# Load Library

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import re
import string
import tensorflow as tf

# Data Preprocessing

Preprocessing Target:

Position:
- Data Scientist

Skills:

- Python
- SQL
- R

Summary:

- **narative**

## Load Data

In [ ]:
dataset = []
with open('/content/drive/MyDrive/Resume Dataset/resume_samples.txt', 'r', encoding='windows-1252') as f:
  for line in f.readlines():
    l = line.strip().split(':::')
    dataset.append(l)
    # experience.append(l[1])
    # summary.append(l[2].replace(u'\xa0', u' '))
        # if a == 4:
        #   break
        # l = line.strip().split(':::')
        # for i in l:
        #   print(i)
        # a+=1

In [ ]:
dataset[2]

['C:\\Workspace\\java\\scrape_indeed\\dba_part_1\\100.html#3',
 'Oracle Database Administrator;Oracle Database Administrator;DATABASES;ORACLE (4 years);ORACLE 10G;SQL;LINUX;RMAN (4 years);DATAGUARD (3 years)',
 'Oracle Database Administrator Oracle <span class="hl">Database</span> <span class="hl">Administrator</span> Oracle Database Administrator - Cognizant hyderabad Carrier Objective:\xa0 To Obtain a Oracle DBA Position in a progressive Company Where I Can Utilise and Enhance my Experience and Knowledge in Constituting Effectively to the Success of the Organization, and also to Improve my Further Technical and Professional Skills.\xa0 Profile Summary:\xa0 • Over 4+ years of Experience as Architecture.\xa0 • Experienced in Installation and Maintaining Highly Critical Databases in any Operational mode on Unix/Linux, Sun Solaris.\xa0 • Experienced in 10g, 11g, 12c databases.\xa0 • Responsible for installing Oracle 11g and 12c in Single Node & Multi node\'s Configure host and listener.\

## Data Cleaning

In [ ]:
def clean_summ(summ, pos):
  summ = summ.replace(u"\\",u" ").replace(u'\xa0', u' ')
  summ = re.sub(r'<span class="hl">', '', summ)
  summ = re.sub(r'<\/span>', '', summ)
  pos = pos.replace(u'\xa0', u' ')
  pos = pos.replace(u"\\",u" ")
  pos = pos.replace(u"(",u"\(")
  pos = pos.replace(u")",u"\)")
  pos = pos.replace(u"?",u"\?")
  pos = re.sub(r'[-+]', '', pos)
  summ = re.sub(r'[-+]', '', summ)
  summ = re.sub(f'{pos}', '', summ,count=3)
  summ = re.sub(r' - ', '', summ, count=1)
  summ = summ.lstrip(' ')
  return summ

# print(clean_summ("Database Administrator <span class=\"hl\">Database</span> <span class=\"hl\">Administrator</span> Database Administrator - Family Private Care LLC Lawrenceville, GA A self-motivated Production SQL Server Database Administrator who possesses\xa0 strong analytical and problem solving skills. My experience includes SQL Server\xa0 2005, 2008 and 2012, 2014, SSIS, as well as clustering, mirroring, and high\xa0 availability solutions in OLTP environments. I am proficient in database backup,\xa0 recovery, performance tuning, maintenance tasks, security, and consolidation.\xa0 I am confident that I would make a beneficial addition to any company. Over the\xa0 course of my career thus far, I have designed databases to fit a variety of needs,\xa0 successfully ensured the security of those databases, problem-solved in order to meet\xa0 both back-end and front-end needs, installed and tested new versions database\xa0 management systems, customized and installed applications and meticulously\xa0 monitored performance for the smoothest front-end experience possible. During my\xa0 5 to 6 years working with databases. Wo","Database Administrator"))

In [ ]:
X_dataset = []
Y_dataset = []
a = 0
for i in dataset:
  # print(a)
  a+=1
  if a%100 == 0:
    print(f"{a} data processed")
  X = ""
  Y = ""
  exp = i[1].split(";")
  summ = clean_summ(i[2], exp[0])
  X+="Experience: "
  exp_lst = []
  for j in exp:
    if "," in j:
      for x in j.split(","):
        x = re.sub('[%s]' % re.escape(string.punctuation.replace("()","")), ' ', x.strip())
        exp_lst.append(x)
      continue
    exp_lst.append(j)
  exp = ", ".join(exp_lst)
  X+=exp
  X_dataset.append(X)
  Y+="Summary: "
  Y+=summ
  Y_dataset.append(Y)

100 data processed
200 data processed
300 data processed
400 data processed
500 data processed
600 data processed
700 data processed
800 data processed
900 data processed
1000 data processed
1100 data processed
1200 data processed
1300 data processed
1400 data processed
1500 data processed
1600 data processed
1700 data processed
1800 data processed
1900 data processed
2000 data processed
2100 data processed
2200 data processed
2300 data processed
2400 data processed
2500 data processed
2600 data processed
2700 data processed
2800 data processed
2900 data processed
3000 data processed
3100 data processed
3200 data processed
3300 data processed
3400 data processed
3500 data processed
3600 data processed
3700 data processed
3800 data processed
3900 data processed
4000 data processed
4100 data processed
4200 data processed
4300 data processed
4400 data processed
4500 data processed
4600 data processed
4700 data processed
4800 data processed
4900 data processed
5000 data processed
5100 data

# Try Modelling

In [ ]:
!pip install transformers torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.2 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.9 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, TFT5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = TFT5ForConditionalGeneration.from_pretrained("google/flan-t5-small")

/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at google/flan-t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


## Tokenizer

In [ ]:
inputs = tokenizer(X_dataset, max_length=128, padding="max_length", truncation=True, return_tensors="tf").input_ids
labels = tokenizer(Y_dataset, max_length=256, padding="max_length", truncation=True, return_tensors="tf").input_ids

In [ ]:
len(labels)

5000

In [ ]:
model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  16449536  
                                                                 
 encoder (TFT5MainLayer)     multiple                  35332800  
                                                                 
 decoder (TFT5MainLayer)     multiple                  41628352  
                                                                 
 lm_head (Dense)             multiple                  16449536  
                                                                 
Total params: 76,961,152
Trainable params: 76,961,152
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.optimizers import Adam

num_epochs = 3
batch_size = 32

model.compile(optimizer=Adam(3e-5))
model.fit(x=inputs, y=labels,epochs=num_epochs, batch_size=batch_size)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


Epoch 1/3
157/157 [==============================] - 231s 1s/step - loss: 3.8373
Epoch 2/3
157/157 [==============================] - 171s 1s/step - loss: 3.7606
Epoch 3/3
157/157 [==============================] - 173s 1s/step - loss: 3.7049


In [ ]:
i = dataset[100]
X = ""
Y = ""
exp = i[1].split(";")
summ = clean_summ(i[2], exp[0])
X+="Experience: "
exp_lst = []
for j in exp:
  if "," in j:
    for x in j.split(","):
      x = re.sub('[%s]' % re.escape(string.punctuation.replace("()","")), ' ', x.strip())
      exp_lst.append(x)
    continue
  exp_lst.append(j)
exp = ", ".join(exp_lst)
X+=exp
X_dataset.append(X)
Y+="Summary: "
Y+=summ
Y_dataset.append(Y)

In [ ]:
test_1 = tokenizer(X, max_length=128, padding="max_length", truncation=True, return_tensors="tf").input_ids

In [ ]:
outputs = model.generate(test_1)

In [ ]:
X

'Experience: F-35 Suitability Analyst, F-35 Database Administrator, Project Lead, Software Engineer'

In [ ]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Summary: F-35 Suitability Analyst F-35 Database Administrator F-35 Database Administrator
